# Install what we need

In [ ]:
!pip install transformers  # Neural network language model
!pip install datasets # Processing dataset
!pip install scikit-learn
!pip install evaluate # Evaluate our model
!pip install accelerate

In [2]:
# google drive를 colab에 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data

### Check data

In [1]:
import pandas as pd

In [4]:
import os 

data_path = os.path.join(os.getcwd(), 'dataset', 'autodrive.csv')

data = pd.read_csv(data_path, encoding='cp949')

In [6]:
print(data.head())
print(data.shape)

   No.    도메인 세부 도메인 의도 (Intention)                                      문장
0    1  차량 제어  운전 제어       일자 따라 가기                        앞 차 뒤에서 똑같이 가 줘.
1    2  주행 정보  주행 기록    자율 주행 주행 기록  최근 일주일 동안 자율 주행할 때 가장 많이 사용했던 기능 알려 줘.
2    3  차량 제어  도어 제어      조수석 차문 열기                      조수석 차 문 여는 거 도와 줘.
3    4  차량 제어  공조 제어         에어컨 제어                        에어컨 온도 밑으로 내려 줘.
4    5  차량 제어  속도 제어       주행 속도 세팅                         차를 오십으로 몰아 주세요.
(3000, 5)


In [7]:
print(data['도메인'].unique())
print(f"도메인 개수 : {len(data['도메인'].unique())}")
print(data['세부 도메인'].unique())
print(f"세부 도메인 개수 : {len(data['세부 도메인'].unique())}")
print(data['의도 (Intention)'].unique())
print(f"의도 개수 : {len(data['의도 (Intention)'].unique())}")

['차량 제어' '주행 정보' '비상 제어' '차선 제어']
도메인 개수 : 4
['운전 제어' '주행 기록' '도어 제어' '공조 제어' '속도 제어' '좌석 제어' '보험사 연락' '자율 주행 가능 여부'
 '차선 변경' '긴급 차량' '자율 주행 시작' '갓길 이동 정차' '차량 비상 정지' '창문 제어' '차선 유지' '속도 확인']
세부 도메인 개수 : 16
['일자 따라 가기' '자율 주행 주행 기록' '조수석 차문 열기' '에어컨 제어' '주행 속도 세팅' '좌석 통풍 켜기'
 '자동 보험사 연락' '전체 차문 열기' '자율 주행 가능 여부 확인' '오른쪽으로 차선 변경' '에스오에스차량 양보'
 '자율 주행 모드 시작' '좌석 온도 내리기' '뒷자리 차문 열기' '일자 추월' '갓길 이동 명령' '좌석 온도 올리기'
 '차선 변경' '히터 제어' '비상 정지 모드 켜기' '전체 창문 닫기' '비상 정지 모드 끄기' '자율 주행 모드 종료'
 '조수석 차문 닫기' '좌석 온도 제어' '조수석 창문 열기' '차선 유지 모드 켜기' '시동석 창문 닫기' '전체 창문 열기'
 '조수석 창문 닫기' '왼쪽으로 차선 변경' '좌석 통풍 끄기' '히터 켜기' '자율 주행 속도 확인' '전체 차문 닫기'
 '뒷자리 차문 닫기' '에어컨 끄기' '시동석 창문 열기' '히터 끄기' '에어컨 켜기' '좌석 통풍 제어'
 '차선 유지 모드 끄기']
의도 개수 : 42


In [8]:
label_dict = {}
idtolabel = {}
for i, label in enumerate(data['의도 (Intention)'].unique()):
  label_dict[label] = i
  idtolabel[i] = label

data['labels'] = data['의도 (Intention)'].map(label_dict)

In [9]:
data.loc[0]

No.                              1
도메인                          차량 제어
세부 도메인                       운전 제어
의도 (Intention)            일자 따라 가기
문장                앞 차 뒤에서 똑같이 가 줘.
labels                           0
Name: 0, dtype: object

### Build dataset

In [10]:
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [11]:
train, test = train_test_split(data, test_size=0.2, random_state=42)
valid, test = train_test_split(test, test_size=0.5, random_state=42)


# test.to_csv('/content/drive/MyDrive/lecture_code/mirae/명령어_차량_대본_원본/test.csv', encoding="cp949")
test.to_csv(os.path.join(os.getcwd(), 'dataset', 'test.csv'), encoding="cp949")

train = Dataset.from_pandas(train, split="train")
valid = Dataset.from_pandas(valid, split="valid")
test = Dataset.from_pandas(test, split="test")

print(train)
print(valid)
print(test)

Dataset({
    features: ['No.', '도메인', '세부 도메인', '의도 (Intention)', '문장', 'labels', '__index_level_0__'],
    num_rows: 2400
})
Dataset({
    features: ['No.', '도메인', '세부 도메인', '의도 (Intention)', '문장', 'labels', '__index_level_0__'],
    num_rows: 300
})
Dataset({
    features: ['No.', '도메인', '세부 도메인', '의도 (Intention)', '문장', 'labels', '__index_level_0__'],
    num_rows: 300
})


In [12]:
print(train)

Dataset({
    features: ['No.', '도메인', '세부 도메인', '의도 (Intention)', '문장', 'labels', '__index_level_0__'],
    num_rows: 2400
})


# Model

https://huggingface.co/models

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-small",
    num_labels=len(data['의도 (Intention)'].unique()), \
    id2label=idtolabel )
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
def tokenize_function(examples):
    return tokenizer(examples["문장"], padding="max_length", truncation=True)

In [14]:
tokenized_train = train.map(tokenize_function, batched=True)
tokenized_valid = valid.map(tokenize_function, batched=True)
print(tokenized_train)
print(tokenized_valid)

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Dataset({
    features: ['No.', '도메인', '세부 도메인', '의도 (Intention)', '문장', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2400
})
Dataset({
    features: ['No.', '도메인', '세부 도메인', '의도 (Intention)', '문장', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 300
})


In [15]:
print(tokenized_train['input_ids'][0])

[0, 12092, 4036, 10214, 4698, 11800, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

# Trainer

https://huggingface.co/docs/transformers/v4.19.2/en/main_classes/trainer#transformers.TrainingArguments

In [16]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [17]:
from transformers import TrainingArguments, Trainer


OUTPUT_DIR = os.path.join(os.getcwd(), 'output')

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,             # output directory
    num_train_epochs=5,                 # total number of training epochs
    per_device_train_batch_size=10,      # batch size per device during training
    per_device_eval_batch_size=10,      # batch size for evaluation
    warmup_steps=1000,                  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                  # strength of weight decay
    logging_dir='./logs',               # directory for storing logs
    logging_steps=200,                  # How often to print logs
    do_train=True,                      # Perform training
    do_eval=True,                       # Perform evaluation
    evaluation_strategy="epoch",        # evalute after eachh epoch
    run_name="v1",                      # experiment name
    seed=3,                             # Seed for experiment reproducibility 3x3
    label_names=['labels']
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,3.504100,2.413044,0.513333
2,2.173300,1.119957,0.740000
3,1.195100,0.627622,0.843333
4,0.655200,0.376296,0.900000
5,0.220400,0.255598,0.930000


TrainOutput(global_step=1200, training_loss=1.3560402647654215, metrics={'train_runtime': 562.8801, 'train_samples_per_second': 21.319, 'train_steps_per_second': 2.132, 'total_flos': 1590742720512000.0, 'train_loss': 1.3560402647654215, 'epoch': 5.0})

# Save model

In [19]:
trainer.save_model()

# Inference

In [20]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")
model = AutoModelForSequenceClassification.from_pretrained(OUTPUT_DIR).to(device)

In [21]:
intent = set(['일자 따라 가기', '자율 주행 주행 기록', '조수석 차문 열기', '에어컨 제어', '주행 속도 세팅', '좌석 통풍 켜기', '자동 보험사 연락', '전체 차문 열기', '자율 주행 가능 여부 확인', '오른쪽으로 차선 변경', '에스오에스차량 양보',
 '자율 주행 모드 시작', '좌석 온도 내리기', '뒷자리 차문 열기', '일자 추월', '갓길 이동 명령', '좌석 온도 올리기', '차선 변경', '히터 제어', '비상 정지 모드 켜기', '전체 창문 닫기', '비상 정지 모드 끄기', '자율 주행 모드 종료', '조수석 차문 닫기', '좌석 온도 제어', '조수석 창문 열기', '차선 유지 모드 켜기', '시동석 창문 닫기', '전체 창문 열기', '조수석 창문 닫기', '왼쪽으로 차선 변경', '좌석 통풍 끄기', '히터 켜기', '자율 주행 속도 확인', '전체 차문 닫기', '뒷자리 차문 닫기', '에어컨 끄기', '시동석 창문 열기', '히터 끄기', '에어컨 켜기', '좌석 통풍 제어',  '차선 유지 모드 끄기'])

In [23]:
import pandas as pd
from datasets import Dataset

test = pd.read_csv(os.path.join(os.getcwd(), 'dataset', 'test.csv'), encoding='cp949')
test['labels'] = test['의도 (Intention)'].map(label_dict)
for i in range(len(test)):
  if test.loc[i]['의도 (Intention)'] in intent:
    pass
    # print(test.loc[i]['의도 (Intention)'])
  else:
    test = test.drop(i)
print(len(test))
test = Dataset.from_pandas(test)


300


In [24]:
results = []
for i in range(test.num_rows):
  inputs = tokenizer(test['문장'][i], padding="max_length", truncation=True, return_tensors="pt").to(device)
  result = model(**inputs)
  logit, id = result['logits'].max(dim=-1)
  results.extend(id.tolist())


In [25]:
correct = 0
for result, answer in zip(results, test['labels']):
  if result == answer:
    correct += 1

score = correct / len(results)

print(score)

0.9066666666666666
